In [ ]:
import pandas as pd
import numpy as np
import os
import random

In [ ]:
os.environ['GGS_DIR'] = '/kaggle/input/gitaggle-settings/gitaggle-settings'
os.environ['SAMPLE_REPO_URL'] = 'https://github.com/dinhtt-randrise/gitaggle-sample.git'
os.environ['DEBUG_ON'] = 'no'
os.environ['USER_NAME'] = 'Dinh Thoai Tran @ randrise.com'
os.environ['USER_EMAIL'] = 'dinhtt@randrise.com'
SAMPLE_REPO_URL = os.environ.get('SAMPLE_REPO_URL', 'https://github.com/dinhtt-randrise/gitaggle-sample.git')
USER_NAME = os.environ.get('USER_NAME', 'Dinh Thoai Tran @ randrise.com')
USER_EMAIL = os.environ.get('USER_EMAIL', 'dinhtt@randrise.com')

In [ ]:
!git clone https://github.com/dinhtt-randrise/gitaggle.git

In [ ]:
%%writefile /kaggle/working/gitaggle/github.py
import os
import argparse

def pat_dir():
    return os.environ.get('GGS_DIR', '/kaggle/input/gitaggle-settings/gitaggle-settings')

def debug_on():
    v = os.environ.get('DEBUG_ON', 'no')
    if v == 'yes':
        return True
    else:
        return False
    
def pat():
    return read_text_file(pat_dir() + '/GitHub-PAT.txt').lstrip().rstrip()

def read_text_file(filename):
    text = ''
    with open(filename, 'r') as f:
        text = f.read()
    return text

def write_text_file(filename, text):
    with open(filename, 'w') as f:
        f.write(text)

def show_help():
    print('=====] gitaggle/github.py [=====')
    print('')
    print('[SYNTAX]')
    print('')
    print('github.py --cmd <command> --work_dir [working_folder] --repo_url [repository_url] --repo_dir [repository_folder] --msg_file [message_file] --user_name [user_name] --user_email [user_email]')
    print('')
    print('==> Commands <==')
    print('')
    print('<help> show this help')
    print('<clone> clone repository')
    print('<commit> commit repository w/ pushing')
    print('<identity> set config (user name / user email)')
    print('')
    
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--cmd", type=str, default='help')
    parser.add_argument("--work_dir", type=str, default=None)
    parser.add_argument("--repo_dir", type=str, default=None)
    parser.add_argument("--msg_file", type=str, default=None)    
    parser.add_argument("--user_name", type=str, default=None)    
    parser.add_argument("--user_email", type=str, default=None)        
    parser.add_argument("--repo_url", type=str, default='https://github.com/dinhtt-randrise/gitaggle.git')    
    args = parser.parse_args()
    if args.cmd == 'clone':
        clone(args.repo_url, args.work_dir)
    elif args.cmd == 'commit':
        commit(args.repo_dir, args.msg_file)
    elif args.cmd == 'identity':
        identity(args.repo_dir, args.user_name, args.user_email)
    else:
        show_help()
    
def clone(repo_url, to_dir = None):
    if to_dir is None:
        cmd = 'git clone ' + repo_url.replace('https://', 'https://' + pat() + '@')
    else:
        cmd = 'cd "' + str(to_dir) + '"' + ' && git clone ' + repo_url.replace('https://', 'https://' + pat() + '@')
    if debug_on():
        print('=> [CMD] ' + cmd)
    os.system(cmd)
    
def commit(repo_dir, msg_file):
    if repo_dir is None or msg_file is None:
        print('=> [E] File or folder is not found!')
        return
    if not os.path.exists(repo_dir) or not os.path.exists(msg_file):
        print('=> [E] File or folder does not exist!')
        return
    cmd = 'cd "' + repo_dir + '" && git add *.* && git commit -F "' + msg_file + '" && git push'
    if debug_on():
        print('=> [CMD] ' + cmd)
    os.system(cmd)
    
def identity(repo_dir, name, email):
    if repo_dir is None:
        print('=> [E] Folder is not found!')
        return
    if not os.path.exists(repo_dir):
        print('=> [E] Folder does not exist!')
        return    
    if name is None:
        print('=> [E] Name is required!')
        return
    if email is None:
        print('=> [E] Email is required!')
        return
    cmd = 'cd "' + repo_dir + '" && git config user.name "' + name + '" && git config user.email "' + email + '"'
    if debug_on():
        print('=> [CMD] ' + cmd)
    os.system(cmd)
    
if __name__ == "__main__":
    main()

In [ ]:
import gitaggle.github as gggh

!mkdir -p /kaggle/working/inline
!mkdir -p /kaggle/working/script

folder_name = SAMPLE_REPO_URL.split('/')[-1].replace('.git', '')
repo_dir_inline = '/kaggle/working/inline/' + folder_name
repo_dir_script = '/kaggle/working/script/' + folder_name

gggh.clone(SAMPLE_REPO_URL, '/kaggle/working/inline')
gggh.write_text_file(repo_dir_inline + '/hello-inline.txt', '[' + str(random.randint(0, 1000)) + '] ' + 'Hello world!')
gggh.write_text_file('/kaggle/working/inline_msg.txt', 'Committed from python function calling')
gggh.identity(repo_dir_inline, USER_NAME, USER_EMAIL)
gggh.commit(repo_dir_inline, '/kaggle/working/inline_msg.txt')


In [ ]:
!python /kaggle/working/gitaggle/github.py --cmd clone --repo_url $SAMPLE_REPO_URL --work_dir /kaggle/working/script
gggh.write_text_file(repo_dir_script + '/hello-script.txt', '[' + str(random.randint(0, 1000)) + '] ' + 'Hello world!')
gggh.write_text_file('/kaggle/working/script_msg.txt', 'Committed from python script calling')
!python /kaggle/working/gitaggle/github.py --cmd identity --repo_dir $repo_dir_script --user_name "$USER_NAME" --user_email "$USER_EMAIL"
!python /kaggle/working/gitaggle/github.py --cmd commit --repo_dir $repo_dir_script --msg_file /kaggle/working/script_msg.txt
